In [ ]:
%env RAY_verbose_spill_logs=0

In [ ]:
import ray

# Define the runtime_env specification
runtime_env = {
    "env_vars":{"HTTP_PROXY": "http://10.78.90.46:80", "HTTPS_PROXY": "http://10.78.90.46:80", "http_proxy": "http://10.78.90.46:80", "https_proxy": "http://10.78.90.46:80"}
}

@ray.remote(runtime_env=runtime_env)
def wrkaround_remote_function():
    try:
        # Install required packages within the function
        import subprocess
        subprocess.run(["pip", "install", "numpy", "pandas"])
       
        # Now you can import and use the packages
        import numpy
        import pandas

        # ... data science engineer and software developers can write code following ...

    except Exception as e:
        print(f"An error occurred: {e}")


def main():
    ray.init(address="ray://kuberay-head-svc.kuberay:10001", ignore_reinit_error=True, logging_level="INFO", runtime_env=runtime_env)

    # Annotate the remote function with runtime_env
    # Install packages asynchronously
    remote_fn_ids = [wrkaround_remote_function.remote() for _ in range(10)]

    # Wait for all tasks to complete
    ray.get(remote_fn_ids)

    # Shutdown the Ray cluster
    ray.shutdown()

if __name__ == "__main__":
    main()